<a id="section-one"></a>
# 📚 Theory

Let's delve into the theory and try to gain a intuition about n-gram language models.

N-Gram models are Statistical(Probabilistic) Language models that aim to assign probabilities to a given sequence of words. Any N-gram is just a sequence of "n" words. For example, "Saurav" is a unigram and "Hi There" is a bigram. 


The task is to find out if we can compute $P(w | h)$ given a word $w$ and some history $h$. One could say that we can compute the probability of a given next word, using all the previous words in the sentence. For example using the last sentence, we could calculate: 

$$\large
P ( \, word \, | \, One \, could \, say \, that \, we \, can \, compute \, the \, probability \, of \, a \, given \, next \,)
$$

---

One such approach could be to use **relative frequency counts** to compute this probability, i.e. ,**Out of the times we saw the history $h$, how many times was it followed by the word $w$**

Or 

$$
P ( \, word \, | \, One \, could \, say \, that \, we \, can \, compute \, the \, probability \, of \, a \, given \, next \,) = \frac{C(\, One \, could \, say \, that \, we \, can \, compute \, the \, probability \, of \, a \, given \, next \, word)}{C(\, One \, could \, say \, that \, we \, can \, compute \, the \, probability \, of \, a \, given \, next \,)}
$$
---

Intuitively it seems infeasible to perform this over an entire corpus; especially it is of a significant a size. This is the motivation behind the N-gram model, instead of using the entire corpus, we approximate this probability using just `n` previous words.

For instance if $w_{1:n}$ represents the sequence of words $w_1w_2...w_n$, then using the chain rule of probability we can write,  


$$\large
P(w_{1:n}) = P(w_1)P(w_2 | w_1)P(w_3 | w_{1:2})...P(w_n|w_{1:n-1})
$$


$$\large
P(w_{1:n}) = \prod_{k=1}^{n}P(w_k | w_{1:k-1})
$$

<a id="bigram-model"></a>
## The Bigram Model ②

A Bigram Model corresponds to a model which approximates the probability of a word given all the previous words $P(w_n|w_{1:n−1})$ by using only the conditional probability of the preceding word $P(w_n|w_{n−1})$. Thus we assume that $P(w_n|w_{1:n−1}) ≈ P(w_n|w_{n−1})$. This approximation is known as the **Markov** approximation. Thus, for the Bigram model, the probability for an entire sequence can be approximated as:

$$\large
P(w_{1:n}) ≈ \prod_{k=1}^{n}P(w_{k}|w_{k−1}) 
$$

<a id="estimation"></a>
## Estimation ⩰

To estimate such probabilities we use the **Maximum Likelihood Estimation (MLE)**. An MLE estimate for the parameters of an n-gram model can be obtained by getting counts from a corpus, and normalizing the counts so that they lie between 0 and 1.

For a Bigram model, the MLE Estimation can be given by:

$$\large
P(w_n | w_{n-1}) \frac{C(w_{n-1}w_n)}{\sum_{w} C(w_{n-1}w)}
$$

---
For the general case of MLE n-gram parameter estimation:

$$\large
P(w_n|w_{n−N+1:n−1}) = \frac{C(w_{n−N+1:n−1}w_n)}{C(w_{n−N+1:n−1})}
$$

In [12]:
import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Basic File Paths
data_dir = "../../data/final/en_US"
file_path = data_dir + "/en_US.twitter.txt"

## nltk settings
#nltk.data.path.append(data_dir)
nltk.download('punkt')

## Opening the File in read mode ("r")
with open(file_path, "r") as f:
    data = f.read()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davideposillipo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We create a simple pipeline function which: 

* splits the datasets by the `\n` character 

* remove leading and trailing spaces 

* drop empty sentences. 

* Tokenize sentences using `nltk.word_tokenize`

In [16]:
def preprocess_pipeline(data) -> 'list':

    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    tokenized = []
    
    for sentence in sentences:
        sentence = sentence.lower()
        token = nltk.word_tokenize(sentence)
        tokenized.append(token)
        
    return tokenized

tokenized_sentences = preprocess_pipeline(data)

In [19]:
len(tokenized_sentences)

2360148

In [20]:
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

train, val = train_test_split(train, test_size=0.25, random_state=42)

As our dataset is quite big, we'll only use those words that appear `k` times in our dataset. In this function, we'll create a frequency dictionary for our vocabulary. 

In [ ]:
def count_the_words(sentences) -> 'dict':
    
  # Creating a Dictionary of counts
  word_counts = {}

  # Iterating over sentences
  for sentence in sentences:
    
    # Iterating over Tokens
    for token in sentence:
    
      # Add count for new word
      if token not in word_counts.keys():
        word_counts[token] = 1
        
      # Increase count by one
      else:
        word_counts[token] += 1
        
  return word_counts

<a id="closed"></a>
## 🔒 Creating a Closed Vocabulary

One of the most essential steps in dealing with Textual data is handling Out-of-vocabulary words. This helps the model to handle words which are not present in the training corpus. First step in this process is to create a `closed_vocabulary`. This function creates a closed vocabulary containing only those words according to the `count_threshold` parameter.

In [ ]:
def handling_oov(tokenized_sentences, count_threshold) -> 'list':

  # Empty list for closed vocabulary
  closed_vocabulary = []

  # Obtain frequency dictionary using previously defined function
  words_count = count_the_words(tokenized_sentences)
    
  # Iterate over words and counts 
  for word, count in words_count.items():
    
    # Append if it's more(or equal) to the threshold 
    if count >= count_threshold :
      closed_vocabulary.append(word)

  return closed_vocabulary

<a id="unk"></a>
## 🤷🏻 Adding UNK Tokens

In this function we'll add `<unk>` tokens, to those words which are not in the `closed_vocabulary` which we just made.

In [ ]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>") -> 'list':

  # Convert Vocabulary into a set
  vocabulary = set(vocabulary)

  # Create empty list for sentences
  new_tokenized_sentences = []
  
  # Iterate over sentences
  for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk> 
    # if the token is absent from the vocabulary
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
    
    # Append sentece to the new list
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences

<a id="final"></a>
## 🧼 Final Cleaning Pipeline

In [ ]:
def cleansing(train_data, test_data, count_threshold):
    
  # Get closed Vocabulary
  vocabulary = handling_oov(train_data, count_threshold)
    
  # Updated Training Dataset
  new_train_data = unk_tokenize(train_data, vocabulary)
    
  # Updated Test Dataset
  new_test_data = unk_tokenize(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary

In [ ]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)

<a id="build"></a>
# 💪🏻 Building The "Model"

This is a helper function, which will come in handy during inference. This function returns a mapping from n-grams to their frequency in the dataset. 

In [ ]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':

  # Empty dict for n-grams
  n_grams = {}
 
  # Iterate over all sentences in the dataset
  for sentence in data:
        
    # Append n start tokens and a single end token to the sentence
    sentence = [start_token]*n + sentence + [end_token]
    
    # Convert the sentence into a tuple
    sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
    m = len(sentence) if n==1 else len(sentence)-1
    
    # Iterate over this length
    for i in range(m):
        
      # Get the n-gram
      n_gram = sentence[i:i+n]
    
      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      # Add n-gram count
      else:
        n_grams[n_gram] = 1
        
  return n_grams

This function calculates the priority for the next word given the prior n-gram. This function also implements k-smoothing which helps account for unseen n-grams. Using the previously defined formula:


$$\large
P(w_n|w_{n−N+1:n−1}) = \frac{C(w_{n−N+1:n−1}w_n)}{C(w_{n−N+1:n−1})}
$$

---

### K-smoothing

But what if we come across a n-gram that wasn't in the training set. Then our denominator would would become zero and our definition of probability will become invalid. Thus, we use k-smoothing, which adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary. This ensures any n-gram with zero count has the same probability of $\frac{1}{|V|}$. Thus, our original estimation get's modified to:

$$\large
P(w_n|w_{n−N+1:n−1}) = \frac{C(w_{n−N+1:n−1}w_n) + k}{C(w_{n−N+1:n−1} + k |V|)}
$$

In [ ]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':

  # Convert the previous_n_gram into a tuple 
  previous_n_gram = tuple(previous_n_gram)
    
  # Calculating the count, if exists from our freq dictionary otherwise zero
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  
  # The Denominator
  denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
  nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero 
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
  num = nplus1_gram_count + k

  # Final Fraction
  prob = num / denom
  return prob

Now, we loop over all the words in the vocabulary and then compute their probabilites using our `prob_for_single_word()` fn.

In [ ]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0) -> 'dict':

  # Convert to Tuple
  previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary
  vocabulary = vocabulary + ["<e>", "<unk>"]

  # Calculate the size of the vocabulary
  vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
  probabilities = {}

  # Iterate over words 
  for word in vocabulary:
    
    # Calculate probability
    probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
    probabilities[word] = probability

  return probabilities

<a id="auto-complete"></a>
# 💬 The Auto-Complete System

Finally, we build our `auto_complete` fn. We simply loop over all the words in the vocabulary assuming that they can be the next word and then return the word with it's probability. 

In [ ]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):

    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]
    
    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = None
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():
        
        if start_with != None: 
            
            if not word.startswith(start_with):
                continue 

        if prob > max_prob: 

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

We can also loop over all the various n-gram models to get multiple suggestions. This function just extends from the previously defined function by **taking multiple n-gram counts** instead of one. This allows us to take unigram, bigram, .. counts into account as well.

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):

    # See how many models we have
    count = len(n_gram_counts_list)
    
    # Empty list for suggestions
    suggestions = []
    
    # IMP: Earlier "-1"
    
    # Loop over counts
    for i in range(count-1):
        
        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        # get suggestions 
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        # Append to list
        suggestions.append(suggestion)
        
    return suggestions

<a id="inference"></a>
# 😊 Inference

Here, we create a list of n-gram counts for a arbitrary range `(1,6)`

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

Let's give it a sample input of "i was about" in a tokenized manner and get multiple suggestions using the above calculated n-gram counts with smoothing-factor, `k` = 1.0 

In [ ]:
previous_tokens = ["i", "was", "about"]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

<a id="misc"></a>
# 🧐 Miscellaneous 

Let's see how many n-grams we have in our corpus.

In [ ]:
print("unigram count:" , len(n_gram_counts_list[0]))
print("bigram count:", len(n_gram_counts_list[1]))
print("trigram count:", len(n_gram_counts_list[2]))
print("quadgram count:", len(n_gram_counts_list[3]))
print("quintgram count:", len(n_gram_counts_list[4]))

In this section, we just export this list to a `.txt` file so that we can use this for inference rather than "training" each time.

In [ ]:
# Storing to file
with open("en_counts.txt", 'wb') as f:
    pickle.dump(n_gram_counts_list, f)

In [ ]:
# Storing to file
with open("vocab.txt", 'wb') as f:
    pickle.dump(vocabulary, f)